# Tailwind CSS with django-tailwind

This guide shows how to set up Tailwind CSS in your Django project using the [django-tailwind](https://django-tailwind.readthedocs.io/) package. This approach is more integrated and does not require Bun or npm.

---

## Step 1: Install django-tailwind

In your virtual environment, install the package:

```sh
pip install django-tailwind
```

Add `tailwind` and `theme` (your custom app, see below) to `INSTALLED_APPS` in `config/settings.py`.

---

## Step 2: Create a Tailwind Theme App

Generate a new Django app for your Tailwind theme (recommended name: `theme`):

```sh
python manage.py startapp theme apps/theme
```
This creates an app at `apps/theme/`. Then, add the correct config in settings:

```python
TAILWIND_APP_NAME = 'theme'
```
Then initialize the Tailwind configuration in that app:


```sh
python manage.py tailwind init
```

---

## Step 3: Configure django-tailwind

In `config/settings.py`, add:

```python
INSTALLED_APPS = [
    ...
    'tailwind',
    'theme',  # your theme app
    ...
]

TAILWIND_APP_NAME = 'theme'
```

---

## Step 4: Custom Tailwind Configuration

Edit `apps/theme/tailwind.config.js` to customize your Tailwind setup. Example:

```js
module.exports = {
  content: [
    '../templates/**/*.html',
    '../../apps/**/*.py',
  ],
  theme: {
    extend: {},
  },
  plugins: [],
}
```

You can also edit `apps/theme/static_src/css/input.css` to include:

```css
@tailwind base;
@tailwind components;
@tailwind utilities;
```

---

## Step 5: Install Node.js (if not already installed)

django-tailwind uses Node.js under the hood. Install it if you haven't:

- [Node.js Download](https://nodejs.org/)

---

## Step 6: Install Tailwind Dependencies

From your project root, run:

```sh
python manage.py tailwind install
```

---

## Step 7: Build Tailwind CSS

For development (auto-reload):

```sh
python manage.py tailwind start
```

For production build:

```sh
python manage.py tailwind build
```

---

## Step 8: Overriding the runserver Command

To automatically run Tailwind in watch mode with the Django server, override the `runserver` command.

Create `apps/core/management/commands/runserver.py` (if not already present):

```python
import subprocess
import sys
import threading
from django.core.management.commands.runserver import Command as RunserverCommand

class Command(RunserverCommand):
    def run_tailwind(self):
        # Use sys.executable for cross-platform compatibility
        self.tailwind_process = subprocess.Popen([sys.executable, 'manage.py', 'tailwind', 'start'])

    def handle(self, *args, **options):
        # Start Tailwind in a background thread
        thread = threading.Thread(target=self.run_tailwind, daemon=True)
        thread.start()
        try:
            super().handle(*args, **options)
        finally:
            # Cleanup Tailwind process on server shutdown
            if hasattr(self, 'tailwind_process') and self.tailwind_process.poll() is None:
                self.tailwind_process.terminate()
```


To just build once without watch mode, you can run:

```sh
python manage.py tailwind build
```
Or override the `runserver` command.
```python
# apps/core/management/commands/runserver.py
import os
from django.core.management.commands.runserver import Command as RunserverCommand
from django.core.management import call_command

class Command(RunserverCommand):
    def handle(self, *args, **options):
        # Build Tailwind CSS before starting the server
        if not os.environ.get('RUN_MAIN') or options.get('use_reloader') is False:
             self.stdout.write(self.style.SUCCESS('Building Tailwind CSS...'))
             try:
                 call_command('tailwind', 'build')
             except Exception as e:
                 self.stdout.write(self.style.WARNING(f'Failed to build Tailwind CSS: {e}'))
        
        
        # Call the original runserver command
        super().handle(*args, **options)
```
Ensure your `core` app is listed before `django.contrib.staticfiles` in `INSTALLED_APPS` to override the default command.

```python
# Auto Discover apps in the project
discovered_apps = []
APPS_DIR = os.path.join(BASE_DIR, 'apps')
for app_name in os.listdir(APPS_DIR):
    app_path = os.path.join(APPS_DIR, app_name)
    if os.path.isdir(app_path) and os.path.isfile(os.path.join(app_path, '__init__.py')):
        try:
            importlib.import_module(app_name)
            discovered_apps.append(app_name)
        except ImportError as e:
            print(f"Could not import app {app_name}: {e}") 

INSTALLED_APPS = [
    *discovered_apps,
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```

---

## Step 9: Integrate Tailwind CSS in Templates

In your base template (e.g., `apps/core/templates/includes/layout.html`):

```html
<head>
    {% load static %}
    <link rel="stylesheet" href="{% static 'theme/css/dist/styles.css' %}">
</head>
```

---

## Step 10: Static Files Settings

In `config/settings.py`, ensure:

```python
STATIC_URL = 'static/'
STATICFILES_DIRS = [
    os.path.join(BASE_DIR, 'static'),
]
```

---

## Step 11: Collect Static Files for Production
In a production environment, ensure that your web server (e.g., Nginx, Apache) is configured to serve static files from the directory specified by `STATIC_ROOT`. You may need to run `python manage.py collectstatic` to gather all static files into the `STATIC_ROOT` directory before deploying your application.
- To collect static files for production, run:
```sh
python manage.py collectstatic
```
This will gather all static files into the directory specified by `STATIC_ROOT`, which your web server can then serve.

- Then configure your web server to serve files from the `STATIC_ROOT` directory.

- If you already don't have a tailwind.css file, create it in static/css/ with the following content:
```css
@tailwind base;
@tailwind components;
@tailwind utilities;
```